<a href="https://colab.research.google.com/github/RahaviSelvarajan/yolov5_colab/blob/main/SBI_YOLO_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
!python --version

##***Mount Google Drive***

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##***Mount Onedrive to get the dataset***

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: /content/onedrive &

**Run the below cell only when onedrive gets disconnected in the middle of training and mount it again using the above command.**

In [ ]:
!fusermount -u /content/onedrive

##***Git Clone YOLOv5-Pytorch model***

In [3]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 12385, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 12385 (delta 0), reused 3 (delta 0), pack-reused 12380
Receiving objects: 100% (12385/12385), 12.17 MiB | 11.68 MiB/s, done.
Resolving deltas: 100% (8543/8543), done.


In [ ]:
!unzip /content/drive/MyDrive/val.zip

Install the required python packages

In [ ]:
%cd yolov5
!pip install -r requirements.txt

##***Training***

In [ ]:
# Copy data configuration file into yolo directory
!cp /content/drive/MyDrive/Yolov5/sleepaid.yaml /content/yolov5/data/

In [ ]:
# Import libraries
import os
import glob
import shutil
from pathlib import Path

In [ ]:
val_imgs = Path('/content/yolov5/data/images/test')
val_labels = Path('/content/yolov5/data/labels/test')

if val_imgs.exists():
  shutil.rmtree(val_imgs)
if val_labels.exists():
  shutil.rmtree(val_labels)

os.mkdir(val_imgs)
os.makedirs(val_labels)  

for folder in os.listdir('/content/val'):
  folder_path = os.path.join('/content/val', folder)
  if not folder.startswith('.'):
    for file in os.listdir(folder_path):
      extension = os.path.splitext(file)[1]
      f_path = os.path.join('/content/val',folder, file)
      if extension == '.txt':
        shutil.copy(f_path, val_labels)
      elif extension == '.jpg':
        shutil.copy(f_path, val_imgs)

In [ ]:
!python val.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights '/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt' --project '/content/drive/MyDrive/Yolov5/runs/final_train' 

In [ ]:
# Get the train and test dataset from different folds
def train_test_split(root_dir, test_fold):

    # Make directory for training and testing images
    train_imgs = Path('/content/yolov5/data/images/train')
    test_imgs = Path('/content/yolov5/data/images/test')

    if train_imgs.exists():
      shutil.rmtree(train_imgs)
    if test_imgs.exists():
      shutil.rmtree(test_imgs)

    os.mkdir(train_imgs)
    os.mkdir(test_imgs)

    # Make directory for target labels
    train_labels = Path('/content/yolov5/data/labels/train')
    test_labels = Path('/content/yolov5/data/labels/test')

    if train_labels.exists():
      shutil.rmtree(train_labels)
    if test_labels.exists():
      shutil.rmtree(test_labels)

    os.makedirs(train_labels)
    os.makedirs(test_labels)

    for j, image in enumerate(glob.iglob(os.path.join('/content/drive/MyDrive/Negatives', "*.jpg"))):
      if j<150:
        shutil.copy(image, train_imgs)
      else:
        shutil.copy(image, test_imgs)

    for i, folder in enumerate(os.listdir(root_dir)):
      folder_path = os.path.join(root_dir, folder)
      file_list = [image for image in glob.glob(os.path.join(folder_path, '*.jpg'))]

      # Get the train folds
      if i != test_fold:
        for image in file_list:
          shutil.copy(image, train_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', train_labels)

      # Get the test fold
      elif i == test_fold:
        for image in file_list:
          shutil.copy(image, test_imgs)
          shutil.copy(os.path.splitext(image)[0] + '.txt', test_labels)

for i in range(1):
  train_test_split('/content/drive/MyDrive/Dataset', i)
  !python train.py --batch-size 8 --img-size 1920 --data 'sleepaid.yaml' --weights 'yolov5s.pt' --epochs 300 --cfg 'yolov5s.yaml'  \
                   --save-period 50 --patience 20 --hyp '/content/drive/MyDrive/Yolov5/hyp.yaml' --project '/content/drive/MyDrive/Yolov5/runs/final_train' 
  !mv /content/drive/MyDrive/Yolov5/runs/final_train/exp /content/drive/MyDrive/Yolov5/runs/final_train/results"$i"

In [ ]:
!python detect.py --source '/content/drive/MyDrive/B-DWU128.mp4' --max-det 1 --conf-thres 0.1 --img-size 1920 --weights '/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt' --save-txt --data '/content/yolov5/data/sleepaid.yaml'

In [ ]:
import cv2
import torch
from datetime import datetime
import pandas as pd

In [ ]:
def load_model():

    model = torch.hub.load('ultralytics/yolov5',
                            'custom', path='/content/drive/MyDrive/Yolov5/runs/final_train/results2/weights/best.pt')
    
    model.max_det = 1
    
    classes = ['B-Left_recovery', 'B-Left_lateral', 'B-Left_tilt', 'B-Supine', 'B-Supine_thorax_with_Left_pelvic_tilt', 
                'B-Supine_thorax_with_Right_pelvic_tilt', 'B-Supine_pelvis_with_Left_thorax_tilt', 
                'B-Supine_pelvis_with_Right_thorax_tilt', 'B-Right_tilt', 'B-Right_lateral', 'B-Right_recovery', 'B-Sitting', 
                'N_B-Left_recovery', 'N_B-Left_lateral', 'N_B-Left_tilt', 'N_B-Supine', 'N_B-Supine_thorax_with_Left_pelvic_tilt', 
                'N_B-Supine_thorax_with_Right_pelvic_tilt', 'N_B-Supine_pelvis_with_Left_thorax_tilt', 
                'N_B-Supine_pelvis_with_Right_thorax_tilt', 'N_B-Right_tilt', 'N_B-Right_lateral', 'N_B-Right_recovery', 'N_B-Sitting']
                    
    return model, classes

    
def detect_objects(video_file, model, classes):

  count = 0 

  cap = cv2.VideoCapture(video_file)
  fps = cap.get(cv2.CAP_PROP_FPS)
  objects = list()
  pt_data = dict()
  while True:
      ret, frame = cap.read()
      if ret:
        result = model(frame)
        labels = result.xyxyn[0][:, -1].cpu().numpy()
        if len(labels) != 0:
          count += 1
          objects.append(classes[int(labels[0])])
        if len(objects) > 2:
          if objects[-1] != objects[-2]:
            pt_data[str(count/fps)] = objects[-2]
            count = 0
      else:
          return pt_data


In [ ]:
model, classes = load_model()

data = detect_objects('/content/drive/MyDrive/B-DWU128.mp4', model, classes)

print(data)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-20 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7074853 parameters, 0 gradients
Adding AutoShape... 


{'10.7': 'B-Left_lateral', '15.1': 'B-Supine', '0.0': 'B-Supine_pelvis_with_Right_thorax_tilt', '0.1': 'B-Right_recovery'}


In [ ]:
df = pd.DataFrame.from_dict(data, orient='index', columns=['Position'])
df.index.name = 'Time spent'
display(df.reset_index())

,Time spent,Position
0,10.7,B-Left_lateral
1,15.1,B-Supine
2,0.0,B-Supine_pelvis_with_Right_thorax_tilt
3,0.1,B-Right_recovery


In [ ]:
nb_data = detect_objects('/content/drive/MyDrive/N_B-DWU128.mp4', model, classes)

df = pd.DataFrame.from_dict(nb_data, orient='index', columns=['Position'])
df.index.name = 'Time spent'
display(df.reset_index())

,Time spent,Position
0,0.6,N_B-Left_tilt
1,5.0,N_B-Left_lateral
2,9.9,N_B-Supine
3,5.1,N_B-Supine_thorax_with_Left_pelvic_tilt
4,0.5,N_B-Supine
5,0.0,N_B-Supine
6,5.6,N_B-Supine_thorax_with_Right_pelvic_tilt
7,0.2,N_B-Supine
8,0.1,N_B-Supine_pelvis_with_Left_thorax_tilt
9,10.6,N_B-Supine
